<a href="https://colab.research.google.com/github/ppojawa/neural-network-course/blob/master/03_keras/01_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Konfiguracja środowiska


In [1]:
# Przygotowanie środowiska do pracy z Tensorflow 2.0.
# Jeśli otrzymasz błąd podczas instalacji Tensorflow uruchom tę komórkę raz jeszcze.

!pip uninstall -y tensorflow
!pip install -q tensorflow==2.0.0

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
ERROR: Operation cancelled by user
^C


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

'2.15.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [2]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [3]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
Total params: 44 (176.00 Byte)
Trainable params: 44 (176.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Dodanie kolejnej warstwy

In [5]:
model.add(Dense(units=2))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
Total params: 54 (216.00 Byte)
Trainable params: 54 (216.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [6]:
from tensorflow.keras.activations import linear

random_data = sorted(np.random.randn(200))
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-2.417708,-2.417708
1,-2.201454,-2.201454
2,-2.191922,-2.191922
3,-1.945960,-1.945960
4,-1.937121,-1.937121


In [7]:
px.line(data, x='data', y='linear', width=800, range_y=[-2, 2])

In [8]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-2.417708,0.081832
1,-2.201454,0.099620
2,-2.191922,0.100478
3,-1.945960,0.124995
4,-1.937121,0.125965


In [9]:
px.line(data, x='data', y='sigmoid', width=800, range_y=[-0.5, 1.5])

In [10]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-2.417708,0.0
1,-2.201454,0.0
2,-2.191922,0.0
3,-1.945960,0.0
4,-1.937121,0.0


In [11]:
px.line(data, x='data', y='relu', width=800, range_y=[-0.5, 1.5])

In [12]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-2.417708,-0.984238
1,-2.201454,-0.975813
2,-2.191922,-0.975353
3,-1.945960,-0.960004
4,-1.937121,-0.959305


In [13]:
px.line(data, x='data', y='tanh', width=800, range_y=[-1.5, 1.5])

In [14]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/))
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [15]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [16]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [17]:
data = np.random.randn(1000, 150)
labels = np.random.randint(2, size=(1000, 1))

print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [18]:
data[:3]

array([[-4.15557175e-01, -4.82186572e-01, -2.81187656e-01,
        -4.87395997e-01, -3.10982274e-01,  8.97088943e-01,
        -9.31953182e-01, -2.87995900e-01,  1.22939547e+00,
         5.19002158e-01,  1.31652940e+00,  8.30591700e-01,
        -1.39980919e+00, -1.79488244e-01, -1.03580133e+00,
         8.51826969e-01,  6.00898661e-01,  1.37463051e+00,
         6.60412501e-01,  7.85483260e-02,  3.26561350e-01,
        -8.89756765e-02, -8.54055564e-02,  8.29148977e-02,
        -8.17360498e-01,  3.29995927e-01,  1.49690056e+00,
         3.13540320e-01, -1.48095071e+00,  7.20677416e-01,
        -1.58998593e+00, -6.07657209e-01, -5.20710661e-01,
         1.65731576e-01,  2.33100603e-01,  1.32775931e+00,
         1.70860404e+00,  1.45682045e+00, -2.69781382e+00,
        -2.11174192e-01,  1.01936918e+00,  3.41492390e-01,
        -2.04868930e+00,  7.93685219e-01,  2.67542955e-01,
        -9.90646074e-01, -6.88679842e-02,  1.22504170e+00,
         4.40111469e-02, -1.63046446e-01,  1.29586765e+0

In [19]:
labels[:10]

array([[0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1]])

In [20]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Epoch 1/20
32/32 [==============================] - 1s 2ms/step - loss: 0.9415 - accuracy: 0.5000
Epoch 2/20
32/32 [==============================] - 0s 2ms/step - loss: 0.7595 - accuracy: 0.5340
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 0.6825 - accuracy: 0.5890
Epoch 4/20
32/32 [==============================] - 0s 2ms/step - loss: 0.6385 - accuracy: 0.6280
Epoch 5/20
32/32 [==============================] - 0s 2ms/step - loss: 0.6032 - accuracy: 0.6770
Epoch 6/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5742 - accuracy: 0.7140
Epoch 7/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5462 - accuracy: 0.7450
Epoch 8/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5214 - accuracy: 0.7780
Epoch 9/20
32/32 [==============================] - 0s 2ms/step - loss: 0.4963 - accuracy: 0.7930
Epoch 10/20
32/32 [==============================] - 0s 2ms/step - loss: 0.4744 - accuracy: 0.8120
Epoch 11/20
32/32 [

In [21]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=30)

Epoch 1/20
34/34 [==============================] - 1s 2ms/step - loss: 0.8560 - accuracy: 0.4610
Epoch 2/20
34/34 [==============================] - 0s 2ms/step - loss: 0.7570 - accuracy: 0.5080
Epoch 3/20
34/34 [==============================] - 0s 2ms/step - loss: 0.6984 - accuracy: 0.5500
Epoch 4/20
34/34 [==============================] - 0s 2ms/step - loss: 0.6527 - accuracy: 0.6100
Epoch 5/20
34/34 [==============================] - 0s 2ms/step - loss: 0.6154 - accuracy: 0.6600
Epoch 6/20
34/34 [==============================] - 0s 2ms/step - loss: 0.5839 - accuracy: 0.6990
Epoch 7/20
34/34 [==============================] - 0s 2ms/step - loss: 0.5557 - accuracy: 0.7440
Epoch 8/20
34/34 [==============================] - 0s 2ms/step - loss: 0.5301 - accuracy: 0.7630
Epoch 9/20
34/34 [==============================] - 0s 2ms/step - loss: 0.5057 - accuracy: 0.7850
Epoch 10/20
34/34 [==============================] - 0s 2ms/step - loss: 0.4813 - accuracy: 0.8100
Epoch 11/20
34/34 [

In [22]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
25/25 [==============================] - 1s 12ms/step - loss: 0.7978 - accuracy: 0.5263 - val_loss: 0.8100 - val_accuracy: 0.5000
Epoch 2/20
25/25 [==============================] - 0s 5ms/step - loss: 0.7050 - accuracy: 0.5738 - val_loss: 0.8048 - val_accuracy: 0.4800
Epoch 3/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6503 - accuracy: 0.6363 - val_loss: 0.8040 - val_accuracy: 0.4700
Epoch 4/20
25/25 [==============================] - 0s 6ms/step - loss: 0.6051 - accuracy: 0.6737 - val_loss: 0.8077 - val_accuracy: 0.4700
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5678 - accuracy: 0.7237 - val_loss: 0.8140 - val_accuracy: 0.4700
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5339 - accuracy: 0.7650 - val_loss: 0.8232 - val_accuracy: 0.4550
Epoch 7/20
25/25 [==============================] - 0s 5ms/step - loss: 0.5025 - accuracy: 0.7788 - val_loss: 0.8338 - val_accuracy: 0.4350
Epoch 8/20
25/25 [=

In [24]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [25]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [26]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [27]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
25/25 [==============================] - 1s 14ms/step - loss: 0.7972 - accuracy: 0.5013 - val_loss: 0.7830 - val_accuracy: 0.4800
Epoch 2/20
25/25 [==============================] - 0s 5ms/step - loss: 0.7095 - accuracy: 0.5638 - val_loss: 0.7714 - val_accuracy: 0.4650
Epoch 3/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6618 - accuracy: 0.6087 - val_loss: 0.7665 - val_accuracy: 0.4600
Epoch 4/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6245 - accuracy: 0.6438 - val_loss: 0.7684 - val_accuracy: 0.4750
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5925 - accuracy: 0.6888 - val_loss: 0.7730 - val_accuracy: 0.4700
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5635 - accuracy: 0.7163 - val_loss: 0.7786 - val_accuracy: 0.4800
Epoch 7/20
25/25 [==============================] - 0s 3ms/step - loss: 0.5374 - accuracy: 0.7688 - val_loss: 0.7850 - val_accuracy: 0.4750
Epoch 8/20
25/25 [=

In [28]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [29]:
model.predict(test_data)

1/1 [==============================] - 0s 136ms/step


array([[0.46966472],
       [0.8111477 ],
       [0.7973784 ],
       [0.42475134],
       [0.11792617]], dtype=float32)

In [43]:
np.argmax(model.predict(test_data), axis=1)

1/1 [==============================] - 0s 20ms/step


array([0, 0, 0, 0, 0])

In [44]:
(model.predict(test_data) > .5).astype(np.int32)

1/1 [==============================] - 0s 25ms/step


array([[0],
       [1],
       [1],
       [0],
       [0]], dtype=int32)

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [45]:
data = np.random.random((1000, 150))
labels = np.random.randint(10, size=(1000, 1))

In [46]:
print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [47]:
data[:3]

array([[0.86875876, 0.84200145, 0.68727188, 0.51572377, 0.75582572,
        0.24890209, 0.68961698, 0.04082641, 0.17372523, 0.2156792 ,
        0.82483207, 0.86116581, 0.81234909, 0.1022298 , 0.44333539,
        0.37237223, 0.06166962, 0.74296856, 0.47201502, 0.29196893,
        0.81210766, 0.01717019, 0.94246893, 0.01870073, 0.72847276,
        0.00724586, 0.09073973, 0.16062505, 0.88306263, 0.43283107,
        0.28312764, 0.9154734 , 0.22705644, 0.69321817, 0.02874669,
        0.08428623, 0.47459132, 0.2561379 , 0.43831613, 0.53311951,
        0.41443187, 0.26461823, 0.80831986, 0.92757608, 0.92347341,
        0.75246625, 0.96764392, 0.97064218, 0.79209995, 0.22094449,
        0.15329496, 0.50734161, 0.02779221, 0.77781521, 0.95291727,
        0.9680781 , 0.30301451, 0.18416418, 0.41216231, 0.59925252,
        0.74558732, 0.15763631, 0.74556273, 0.33110698, 0.38213305,
        0.48877187, 0.67578   , 0.19430735, 0.2002467 , 0.75149399,
        0.47288783, 0.4697341 , 0.98871027, 0.26

In [48]:
labels[:10]

array([[1],
       [2],
       [8],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2]])

In [49]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10)
labels

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [50]:
labels[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [51]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 11ms/step - loss: 2.3775 - accuracy: 0.0988 - val_loss: 2.4001 - val_accuracy: 0.1150
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 2.3343 - accuracy: 0.1000 - val_loss: 2.3820 - val_accuracy: 0.0950
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 2.3130 - accuracy: 0.1225 - val_loss: 2.3791 - val_accuracy: 0.0850
Epoch 4/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2984 - accuracy: 0.1250 - val_loss: 2.3842 - val_accuracy: 0.0850
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2731 - accuracy: 0.1600 - val_loss: 2.3981 - val_accuracy: 0.1100
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2699 - accuracy: 0.1312 - val_loss: 2.3870 - val_accuracy: 0.0900
Epoch 7/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2522 - accuracy: 0.1637 - val_loss: 2.3910 - val_accuracy: 0.0850
Epoch 8/30
25/25 [=

In [54]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 25ms/step


array([[0.0380523 , 0.13329665, 0.25665566, 0.06641404, 0.09022842,
        0.03829172, 0.26083717, 0.04048862, 0.04487998, 0.03085542],
       [0.05484024, 0.11270557, 0.15380184, 0.06649917, 0.08436232,
        0.07766022, 0.2262202 , 0.09899862, 0.0457454 , 0.07916648],
       [0.08049964, 0.04567045, 0.06937389, 0.13422921, 0.11140914,
        0.10244563, 0.06209608, 0.19293012, 0.1017898 , 0.09955598],
       [0.03514135, 0.16363695, 0.1555697 , 0.13380364, 0.17016411,
        0.05041424, 0.08671811, 0.09282511, 0.04046537, 0.07126129],
       [0.05617712, 0.04183197, 0.10341062, 0.08044486, 0.06461475,
        0.10555308, 0.06771812, 0.27918497, 0.11976185, 0.08130256],
       [0.03020177, 0.09911946, 0.24184161, 0.16373031, 0.07503326,
        0.05009334, 0.05940568, 0.05207068, 0.09984632, 0.12865765],
       [0.08659901, 0.037746  , 0.09988257, 0.19655327, 0.05448239,
        0.10059409, 0.10243575, 0.03088001, 0.06498836, 0.22583865],
       [0.05442658, 0.11909361, 0.1188493

In [55]:
# model.predict_classes(test_data)
predict_x=model.predict(test_data)
np.argmax(predict_x,axis=1)

1/1 [==============================] - 0s 35ms/step


array([6, 6, 7, 4, 7, 2, 9, 9, 6, 0])

### <a name='a6'></a> Przykład - regresja

In [56]:
data = np.random.random((1000, 150))
labels = 50 * np.random.random(1000)

In [57]:
data[:3]

array([[0.56534481, 0.63125787, 0.32269793, 0.67287387, 0.54429754,
        0.64271098, 0.3309149 , 0.77495082, 0.84104937, 0.50828262,
        0.95559279, 0.32568072, 0.10414149, 0.20401343, 0.18867935,
        0.93452364, 0.6721967 , 0.51405773, 0.8137797 , 0.39318277,
        0.50535576, 0.294447  , 0.74125326, 0.26669046, 0.04548281,
        0.80583323, 0.53771878, 0.58399853, 0.36729359, 0.34289905,
        0.0757532 , 0.63022122, 0.43962627, 0.40690796, 0.93010875,
        0.35940715, 0.91834474, 0.96903036, 0.02553611, 0.22441677,
        0.64878697, 0.40507888, 0.56576618, 0.23798933, 0.15336744,
        0.0556679 , 0.73694091, 0.11778549, 0.31465891, 0.42935904,
        0.63507612, 0.17905778, 0.99505463, 0.54380935, 0.306337  ,
        0.9269119 , 0.39251058, 0.17151046, 0.83544998, 0.71643151,
        0.91237566, 0.49174161, 0.0210837 , 0.44594518, 0.42458785,
        0.55193255, 0.33649011, 0.92391278, 0.37212425, 0.52417018,
        0.70260294, 0.78731006, 0.24595452, 0.48

In [58]:
labels[:10]

array([38.42957069, 12.14117749, 26.39300084,  3.67010881, 37.4750839 ,
       49.66545631, 42.21636036, 42.78007891, 37.75596061, 14.97195171])

In [59]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 12ms/step - loss: 570.8131 - val_loss: 353.6159
Epoch 2/30
25/25 [==============================] - 0s 5ms/step - loss: 291.3803 - val_loss: 222.4584
Epoch 3/30
25/25 [==============================] - 0s 5ms/step - loss: 214.6893 - val_loss: 208.2888
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 207.8956 - val_loss: 208.3669
Epoch 5/30
25/25 [==============================] - 0s 6ms/step - loss: 207.1697 - val_loss: 208.1478
Epoch 6/30
25/25 [==============================] - 0s 5ms/step - loss: 206.6748 - val_loss: 209.8690
Epoch 7/30
25/25 [==============================] - 0s 5ms/step - loss: 205.4790 - val_loss: 208.1397
Epoch 8/30
25/25 [==============================] - 0s 4ms/step - loss: 205.0888 - val_loss: 209.6721
Epoch 9/30
25/25 [==============================] - 0s 4ms/step - loss: 205.2133 - val_loss: 208.2231
Epoch 10/30
25/25 [==============================] - 0s 4ms/step - loss: 204.2621

In [60]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 9ms/step - loss: 19.9390 - mse: 575.4040 - val_loss: 15.6062 - val_mse: 368.7408
Epoch 2/30
25/25 [==============================] - 0s 3ms/step - loss: 14.6369 - mse: 306.0770 - val_loss: 12.8869 - val_mse: 232.9654
Epoch 3/30
25/25 [==============================] - 0s 4ms/step - loss: 12.8215 - mse: 219.9304 - val_loss: 12.5907 - val_mse: 207.2681
Epoch 4/30
25/25 [==============================] - 0s 3ms/step - loss: 12.5470 - mse: 207.8455 - val_loss: 12.6865 - val_mse: 209.8238
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 12.5259 - mse: 208.0154 - val_loss: 12.6450 - val_mse: 208.3260
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 12.5182 - mse: 207.0252 - val_loss: 12.6897 - val_mse: 209.7694
Epoch 7/30
25/25 [==============================] - 0s 4ms/step - loss: 12.5054 - mse: 207.2789 - val_loss: 12.7358 - val_mse: 211.3631
Epoch 8/30
25/25 [==============================

In [ ]:
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[24.40516 ],
       [27.921484],
       [23.890505],
       [23.812803],
       [26.02559 ],
       [24.233002],
       [23.821136],
       [22.760138],
       [27.158115],
       [26.908585]], dtype=float32)